In [42]:
from collections import defaultdict, Counter
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.metrics import make_scorer, matthews_corrcoef
import scipy.sparse as sp
from scipy.sparse import csr_matrix

In [55]:
def load_data_as_lines(path):
    r""" Open text file by path and read all lines """
    with open(path, "r") as fh:
        lines = fh.readlines()
        
    # transform docs into lists of words
    raw_lines = [l.split() for l in lines]

    return raw_lines

def to_data_and_label(raw_lines):
    r""" Split training data and label from raw lines """
    y = list(map(lambda x: int(x[0]), raw_lines))
    x = list(map(lambda x: x[1:], raw_lines))
    return (x, y)

def save_result_as_file(prediction, file_name="prediction.dat"):
    r""" Save the predicted result as a new file """
    file_content = "\n".join(list(map(str, prediction)))
    with open(file_name, "w") as fd:
        fd.write(file_content) 

In [16]:
def kmer(name, k=3):
    r""" Given a name and parameter k, return the vector of k-mers associated with the name
    """
    name = name.lower()
    name_len = len(name)
    v = []
    for i in range(name_len - k + 1):
        v.append(name[i:i+k])
    
    return v

In [37]:
# Create sparse matrices, which requires aggregration of term IDs for both training and test data.

def calc_term_ids(docs):
    r""" The docs should be the combination of training set and test set."""
    term_ids = {}
    curr_term_id = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in term_ids:
                term_ids[w] = curr_term_id
                curr_term_id += 1
    return (term_ids, nnz)

def build_sparse_matrix(docs, term_ids = {}, nnz = 0):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    ncols = len(term_ids)
    assert(ncols != 0)

    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows + 1, dtype=np.int)
    row_id = 0  # document ID / row counter
    acc = 0  # non-zero counter

    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k, _ in cnt.most_common())
        curr_doc_len = len(keys)
        for i, key in enumerate(keys):
            ind[acc + i] = term_ids[key]
            val[acc + i] = cnt[key]
        ptr[row_id + 1] = ptr[row_id] + curr_doc_len
        acc += curr_doc_len
        row_id += 1

    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

In [33]:
def csr_l2normalize(mat, copy=False):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [5]:
# Read files
train_lines = load_data_as_lines("train.dat")
# Split 
x_train, y_train = to_data_and_label(train_lines)

In [56]:
# Read files
x_test = load_data_as_lines("test.dat")

In [57]:
# Text preprocessing
print("Preprocessing documents...")
docs_train = list(map(lambda x: kmer(x[0], k=3), x_train))
docs_test = list(map(lambda x: kmer(x[0], k=3), x_test))

Preprocessing documents...


In [43]:
term_ids, nnz = calc_term_ids([*docs_train])
mat_train = build_sparse_matrix(docs_train, term_ids, nnz)
csr_l2normalize(mat_train)

In [35]:
def splitData(mat, cls, fold=1, d=10):
    r""" Split the matrix and class info into train and test data using d-fold hold-out
    """
    n = mat.shape[0]
    r = int(np.ceil(n*1.0/d))
    mattr = []
    clstr = []
    # split mat and cls into d folds
    for f in range(d):
        if f+1 != fold:
            mattr.append( mat[f*r: min((f+1)*r, n)] )
            clstr.extend( cls[f*r: min((f+1)*r, n)] )
    # join all fold matrices that are not the test matrix
    train = sp.vstack(mattr, format='csr')
    # extract the test matrix and class values associated with the test rows
    test = mat[(fold-1)*r: min(fold*r, n), :]
    clste = cls[(fold-1)*r: min(fold*r, n)]

    return train, clstr, test, clste

In [59]:
def classify(x, train, clstr):
    r""" Classify vector x using kNN and majority vote rule given training data and associated classes
    """
    # find nearest neighbors for x
    dots = x.dot(train.T)
    sims = list(zip(dots.indices, dots.data))
    if len(sims) == 0:
        # could not find any neighbors
        return '+' if np.random.rand() > 0.5 else '-'
    sims.sort(key=lambda x: x[1], reverse=True)
    tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(2)
    if len(tc) < 2 or tc[0][1] > tc[1][1]:
        # majority vote
        return tc[0][0]
    # tie break
    tc = defaultdict(float)
    for s in sims[:k]:
        tc[clstr[s[0]]] += s[1]
    return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]

In [72]:
def classifyNames(cls, c=3, k=3, d=10):
    def classify(x, train, clstr):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = x.dot(train.T)
        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return 1 if np.random.rand() > 0.5 else -1
        sims.sort(key=lambda x: x[1], reverse=True)
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(2)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]
        
    macc = 0.0
    for f in range(d):
        # split data into training and testing
        train, clstr, test, clste = splitData(mat_train, cls, f+1, d)
        # predict the class of each test sample
        clspr = [ classify(test[i,:], train, clstr) for i in range(test.shape[0]) ]
        
        macc += matthews_corrcoef(clste, clspr)
        
    return macc/d

In [52]:
max_c = 4
max_k = 500

max_acc = 0
max_acc_c = 0
max_acc_k = 0
history = [[] for i in range(max_k)]

# for c in range(max_c):
#     for k in range(max_k):
#         accuracy = classifyNames(names, cls, c=c+1, k=k+1)
#         history[c].append(accuracy)
#         if accuracy > max_acc:
#             max_acc = accuracy
#             max_acc_c = c + 1
#             max_acc_k = k + 1


for k in range(max_k):
    accuracy = classifyNames(y_train, k=k+1)
    history[k].append(accuracy)
    if accuracy > max_acc:
        max_acc = accuracy
        max_acc_k = k + 1
            
print("c=%d, k=%d, accuracy: %f" % (max_acc_c, max_acc_k, max_acc))

c=0, k=20, accuracy: 0.698850


In [58]:
term_ids2, nnz2 = calc_term_ids([*docs_train, *docs_test])
mat_train2 = build_sparse_matrix(docs_train, term_ids2, nnz2)
mat_test2 = build_sparse_matrix(docs_test, term_ids2, nnz2)
csr_l2normalize(mat_train2)
csr_l2normalize(mat_test2)

In [73]:
def predict(test_data, train_data, labels, k=1):
    r""" Predict the label of test data based on the given
    labeled training data using k-nearest neighbor classifier.
    """
    predictions = []
    
    similarity = train_data.dot(test_data.T).todense()
    top_k_idx = np.argpartition(similarity, -k, axis=0)[-k:,:]
    to_labels = np.vectorize(lambda idx: labels[idx])
    top_k_label = to_labels(top_k_idx)
    
    n_test_col = similarity.shape[1]

    for col in range(n_test_col):
        train_tags = top_k_label[:,col].flatten().tolist()[0]
        train_indices = top_k_idx[:,col].flatten().tolist()[0]

        # Select the maximum aggregated similarity
        weights = {}
        for tag, row in zip(train_tags, train_indices):
            if tag in weights:
                weights[tag]["count"] += 1
                weights[tag]["value"] += similarity[row, col]
            else:
                weights[tag] = {
                    "count": 1,
                    "value": similarity[row, col]
                }
                
        result = max(weights.items(), key=lambda x: (x[1]["count"], x[1]["value"]))[0]
        predictions.append(result)

    return predictions

In [74]:
y_test = predict(mat_test2, mat_train2, y_train, k=20)
save_result_as_file(y_test, "knn20-kmer3.dat")